In [96]:
import pandas as pd
import numpy as np
import os
import math

In [61]:
# path
path = r"C:\Users\USYS671257\OneDrive - WSP O365\21_31000110.002_Chattanooga TPO Model\model outputs"

# inputs
countycorr = pd.read_csv(os.path.join(path, "county_districts_chattanooga.csv"))
tripdata  = pd.read_csv(os.path.join(path, "_trip_2.dat"), sep = '\t')
hhdata  = pd.read_csv(os.path.join(path, "_household_2.dat"), sep = '\t')
perdata = pd.read_csv(os.path.join(path, "_person_2.dat"), sep = '\t')

# parameters
excludeChildren5 = True

In [92]:
def prep_tripdata(tripdata, perdata):

    perdata = perdata[["hhno","pno","pptyp","psexpfac"]]
    tripdata = pd.merge(tripdata, perdata, on=["hhno","pno"], how="left")
    if excludeChildren5:
        tripdata = tripdata[tripdata["pptyp"]<8]
    tripdata["dephr"] = (tripdata["deptm"]/100).apply(math.trunc)
    tripdata["depmin"] = tripdata["deptm"]-tripdata["dephr"]*100

    if len(tripdata[tripdata["deptm"]>60]["deptm"])>0:
        tripdata["deptime"] = tripdata["deptm"]/60
        tripdata["arrtime"] = tripdata["arrtm"]/60
        tripdata["durdest"] = tripdata["endacttm"]-tripdata["arrtm"]
    else:
        tripdata["deptime"] = tripdata["dephr"]+tripdata["depmin"]/60
        tripdata["arrhr"] = (tripdata["arrtm"]/100).apply(math.trunc)
        tripdata["arrmin"] = tripdata["arrtm"]-tripdata["arrhr"]*100
        tripdata["arrtime"] = tripdata["arrhr"]+tripdata["arrmin"]/60
        tripdata["durdest"] = (
                               ((tripdata["endacttm"]/100).apply(math.trunc)-(tripdata["arrtm"]/100).apply(math.trunc))*60
                               +(tripdata["endacttm"]-(tripdata["endacttm"]/100).apply(math.trunc)*100)
                               -(tripdata["arrtm"]-(tripdata["arrtm"]/100).apply(math.trunc)*100)
                               )

    tripdata["durdest"] = np.where(tripdata.durdest<0,tripdata.durdest+1440,tripdata.durdest)   
    tripdata["durdest"] = tripdata["durdest"]/60

    tripdata = tripdata.sort_values(by=["hhno","pno","tour","half"], ascending=True)
    tripdata["maxtripno"] = tripdata.groupby(["hhno","pno","tour","half"])["tseg"].transform('max')

    cats = [0] + np.arange(3, 28.5, 0.5).tolist()
    tripdata["arrtimecat"] = pd.cut(tripdata["arrtime"], 
                                        bins=cats,  
                                        right=True,
                                        labels=cats[:-1])
    tripdata["deptimecat"] = pd.cut(tripdata["deptime"], 
                                        bins=cats,  
                                        right=True,
                                        labels=cats[:-1])
    cats = [0] + np.arange(0.5, 24.5, 0.5).tolist()
    tripdata["durdestcat"] = pd.cut(tripdata["durdest"], 
                                        bins=cats,  
                                        right=True,
                                        labels=cats[:-1])

    tripdata["arrflag"] = 0
    tripdata["depflag"] = 0
    tripdata["durflag"] = 0

    tripdata.loc[(tripdata["half"]==1) & (tripdata["tseg"]!=tripdata["maxtripno"]), "arrflag"] = 1
    tripdata.loc[(tripdata["half"]==2) & (tripdata["tseg"]!=1), "depflag"] = 1
    tripdata.loc[(tripdata["tseg"]<tripdata["maxtripno"]), "durflag"] = 1

    return tripdata
    
    

def summary_trip_tod(dstripdata, sum_by_var, flag):
    
    """  
        Work 1 
        School 2 
        Escort 3
        Personal_Business 4
        Shop 5
        Meal 6
        SocRec 7   
    """
    
    if flag is None:
        cats = [0] + np.arange(3, 28.5, 0.5).tolist()
    else:    
        dstripdata = dstripdata[dstripdata[flag]==1]
        if flag == "durflag":
            cats = [0] + np.arange(0.5, 24.5, 0.5).tolist()
        else:
            cats = [0] + np.arange(3, 28.5, 0.5).tolist()
    
    index_order = cats[:-1]
    column_order = np.arange(1, 8, 1).tolist()
    summary = (dstripdata.groupby([sum_by_var,"dpurp"])["psexpfac"].
                          sum().
                          reset_index().
                          pivot_table(values='psexpfac', 
                                     index=sum_by_var,
                                     columns="dpurp",
                                     fill_value=0).
                         reindex(index=index_order,
                                 columns=column_order).
                         fillna(0))
    return summary

In [72]:
# clean up trip data
dstripdata = prep_tripdata(tripdata, perdata)

In [109]:
# Arrival_Time_Outbound_Half_Tour
summary_trip_tod(dstripdata, "arrtimecat", flag="arrflag")

dpurp,1,2,3,4,5,6,7
arrtimecat,,,,,,,
0.0,6.0,0.0,21.0,47.0,29.0,1.0,9.0
3.0,86.0,2.0,491.0,357.0,207.0,54.0,46.0
3.5,120.0,4.0,629.0,401.0,279.0,68.0,67.0
4.0,138.0,5.0,669.0,520.0,316.0,76.0,106.0
4.5,156.0,13.0,723.0,615.0,365.0,103.0,128.0
5.0,248.0,15.0,844.0,798.0,402.0,119.0,149.0
5.5,274.0,18.0,1027.0,1031.0,563.0,157.0,195.0
6.0,963.0,81.0,4290.0,3251.0,1695.0,431.0,684.0
6.5,946.0,68.0,3721.0,2804.0,1296.0,267.0,696.0


In [83]:
# Depart_Time_Return_Half_Tour
summary_trip_tod(dstripdata, "deptimecat", flag="depflag")

dpurp,1,2,3,4,5,6,7
deptimecat,,,,,,,
0.0,67.0,3.0,107.0,88.0,72.0,11.0,26.0
3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.5,0.0,0.0,1.0,2.0,4.0,0.0,1.0
4.0,1.0,0.0,1.0,6.0,2.0,1.0,0.0
4.5,0.0,0.0,3.0,16.0,13.0,1.0,1.0
5.0,3.0,0.0,1.0,19.0,7.0,1.0,1.0
5.5,10.0,0.0,5.0,13.0,4.0,1.0,5.0
6.0,9.0,0.0,2.0,15.0,11.0,0.0,3.0
6.5,11.0,0.0,8.0,38.0,28.0,2.0,4.0


In [87]:
# Duration
summary_trip_tod(dstripdata, "durdestcat", flag="durflag")

dpurp,1,2,3,4,5,6,7
durdestcat,,,,,,,
0.0,18493.0,1231.0,43695.0,91289.0,64408.0,13160.0,15763.0
0.5,10507.0,656.0,19437.0,45860.0,31932.0,6835.0,8179.0
1.0,3769.0,276.0,6166.0,17026.0,11820.0,2669.0,3453.0
1.5,2252.0,210.0,3231.0,9833.0,6927.0,1634.0,2220.0
2.0,1355.0,145.0,1689.0,5413.0,3755.0,895.0,1507.0
2.5,1009.0,128.0,1155.0,3952.0,2686.0,728.0,1325.0
3.0,759.0,102.0,697.0,2771.0,1821.0,501.0,1142.0
3.5,567.0,89.0,431.0,2037.0,1294.0,425.0,978.0
4.0,460.0,85.0,294.0,1430.0,960.0,296.0,870.0


In [93]:
# Arrival_Time_All
summary_trip_tod(dstripdata, "arrtimecat", flag=None)

dpurp,1,2,3,4,5,6,7
arrtimecat,,,,,,,
0.0,398.0,5.0,588.0,599.0,467.0,365.0,291.0
3.0,679.0,2.0,660.0,495.0,219.0,63.0,109.0
3.5,1145.0,5.0,814.0,562.0,330.0,83.0,122.0
4.0,1481.0,5.0,826.0,668.0,365.0,93.0,172.0
4.5,1690.0,13.0,896.0,780.0,425.0,128.0,177.0
5.0,1866.0,15.0,963.0,939.0,464.0,163.0,195.0
5.5,2203.0,37.0,1191.0,1183.0,639.0,231.0,238.0
6.0,11308.0,1049.0,5153.0,4024.0,1864.0,532.0,1001.0
6.5,11996.0,1185.0,4576.0,3596.0,1625.0,380.0,990.0


In [94]:
# Depart_Time_All
summary_trip_tod(dstripdata, "deptimecat", flag=None)

dpurp,1,2,3,4,5,6,7
deptimecat,,,,,,,
0.0,824.0,6.0,745.0,719.0,462.0,347.0,300.0
3.0,862.0,5.0,704.0,541.0,262.0,69.0,116.0
3.5,1455.0,13.0,847.0,659.0,359.0,94.0,155.0
4.0,1639.0,35.0,881.0,704.0,394.0,108.0,166.0
4.5,1985.0,100.0,957.0,949.0,468.0,139.0,204.0
5.0,2481.0,216.0,1081.0,1143.0,565.0,205.0,259.0
5.5,7582.0,738.0,2734.0,2551.0,1193.0,399.0,567.0
6.0,12082.0,1668.0,4977.0,3807.0,1783.0,471.0,1018.0
6.5,20365.0,6675.0,6441.0,5473.0,2440.0,562.0,1432.0
